# DSU Worker (Kaggle)

Run all: sets up GPU wheels and launches the watcher. Uses Kaggle's /kaggle/working as the DSU root.


In [ ]:
# One-click venv or target-site (fallback) + CUDA-correct installs (Kaggle)
import os, sys, subprocess, pathlib, re, json
BASE = pathlib.Path('/kaggle/working/M4L-Demucs')
VENV = BASE / '.venv'
PY = VENV / 'bin' / 'python'
SITE_TARGET = BASE / '.py-site' / 'site-packages'
PMARK = BASE / 'PY_MODE.json'
BASE.mkdir(parents=True, exist_ok=True)

print('Kaggle bootstrap. BASE=', BASE)
print(subprocess.getoutput('nvidia-smi'))

mode = 'venv'
py_exec = None
site_dir = ''

# Try isolated venv first; if it fails, fall back to --target site-packages + PYTHONPATH
try:
    if not VENV.exists():
        subprocess.check_call([sys.executable, '-m', 'venv', str(VENV)])
        subprocess.check_call([str(PY), '-m', 'pip', 'install', '--upgrade', 'pip', 'setuptools', 'wheel'])
    py_exec = str(PY)
except Exception as e:
    print('venv failed, using --target site-packages fallback:', e)
    mode = 'target'
    site_dir = str(SITE_TARGET)
    os.makedirs(site_dir, exist_ok=True)
    py_exec = sys.executable

# Detect CUDA and choose torch index/spec
smi = subprocess.getoutput('nvidia-smi')
m = re.search(r'CUDA Version:\s*([0-9.]+)', smi)
if not m:
    raise RuntimeError('GPU not detected. In Kaggle: Settings → Accelerator → GPU, then Run all.')
ver = m.group(1)
if ver.startswith('12.6'):
    fl, torch_ver, ta_ver, idx = ('cu126','2.8.0','2.8.0','https://download.pytorch.org/whl/cu126')
elif ver.startswith('12.4'):
    fl, torch_ver, ta_ver, idx = ('cu124','2.5.1','2.5.1','https://download.pytorch.org/whl/cu124')
elif ver.startswith('12.'):
    fl, torch_ver, ta_ver, idx = ('cu126','2.8.0','2.8.0','https://download.pytorch.org/whl/cu126')
elif ver.startswith('11.8'):
    fl, torch_ver, ta_ver, idx = ('cu118','2.1.2','2.1.2','https://download.pytorch.org/whl/cu118')
else:
    fl, torch_ver, ta_ver, idx = ('cu126','2.8.0','2.8.0','https://download.pytorch.org/whl/cu126')

print('CUDA flavor:', fl)

if mode == 'venv':
    # Torch/torchaudio strictly from CUDA index inside venv
    subprocess.check_call([py_exec, '-m', 'pip', 'install', '--extra-index-url', idx,
                           f'torch=={torch_ver}+{fl}', f'torchaudio=={ta_ver}+{fl}'])
    # Demucs and deps inside venv (no global bleed)
    subprocess.check_call([py_exec, '-m', 'pip', 'install', 'demucs==4.0.1', 'torchcodec', 'ffmpeg-python', 'numpy==2.3.4'])
    # Verify
    subprocess.run([py_exec, '-c', "import torch, demucs; print('Boot OK.', 'torch', torch.__version__, 'cuda', torch.cuda.is_available(), 'demucs', demucs.__version__)"] , check=True)
else:
    # Target install into site_dir and use PYTHONPATH
    subprocess.check_call([py_exec, '-m', 'pip', 'install', '--extra-index-url', idx,
                           '--target', site_dir, f'torch=={torch_ver}+{fl}', f'torchaudio=={ta_ver}+{fl}'])
    subprocess.check_call([py_exec, '-m', 'pip', 'install', '--target', site_dir,
                           'demucs==4.0.1', 'torchcodec', 'ffmpeg-python', 'numpy==2.3.4'])
    env = os.environ.copy()
    env['PYTHONPATH'] = site_dir + (os.pathsep + env.get('PYTHONPATH',''))
    subprocess.run([py_exec, '-c', "import torch, demucs; import os; print('Boot OK.', 'torch', torch.__version__, 'cuda', torch.cuda.is_available(), 'demucs', demucs.__version__, 'mode', 'target')"], check=True, env=env)

# Persist run mode for next cell
PMARK.write_text(json.dumps({'mode': mode, 'py': py_exec, 'site': site_dir, 'base': str(BASE)}))
print('PY_MODE:', PMARK.read_text())


In [ ]:
# Launch watcher (Kaggle)
import urllib.request, pathlib, os, json, subprocess
BASE = pathlib.Path('/kaggle/working/M4L-Demucs')
w = BASE/'colab_watcher.py'
urllib.request.urlretrieve('https://raw.githubusercontent.com/VSTOPIA/Doctor-Sample-Unit-DSU/main/colab_watcher.py', w)
print('Watcher saved to', w)

# Load runtime mode from previous cell
pmark = BASE/'PY_MODE.json'
mode = 'venv'
py_exec = 'python'
site_dir = ''
try:
    data = json.loads(pmark.read_text())
    mode = data.get('mode', 'venv')
    py_exec = data.get('py', 'python')
    site_dir = data.get('site', '')
except Exception:
    pass

os.environ['DSU_ROOT'] = str(BASE)
os.environ['DSU_REMOTE_JOBS_URL'] = 'https://raw.githubusercontent.com/VSTOPIA/Doctor-Sample-Unit-DSU/main/remote_jobs.jsonl'
print('DSU_ROOT=', os.environ['DSU_ROOT'])
print('DSU_REMOTE_JOBS_URL=', os.environ['DSU_REMOTE_JOBS_URL'])

if mode == 'target' and site_dir:
    env = os.environ.copy()
    env['PYTHONPATH'] = site_dir + (os.pathsep + env.get('PYTHONPATH',''))
    subprocess.call([py_exec, str(w)], env=env)
else:
    subprocess.call([py_exec, str(w)])
